In [59]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../..")
from functools import partial
from typing import Literal

import uproot
import numpy as np
import awkward as ak
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
import hist

from analyzer.dumperReader.reader import *
from analyzer.driver.fileTools import *
from analyzer.driver.computations import *
from analyzer.computations.CPToSupercls import CPToSuperclusterProperties
from analyzer.computations.tracksters import tracksters_seedProperties, CPtoTrackster_properties
from analyzer.energy_resolution.fit import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
reader = DumperReader("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/reco-v8-mustache/ticlDumper_mustache_1.root")

In [73]:
reco_sc = reader.fileDir["superclustering"].arrays()
reco_sc

<Array [{run_: 1, ...}, ..., {run_: 1, ...}] type='100 * {run_: uint32, lum...'>

In [74]:
reco_sc.recoSuperCluster_constituentTs

<Array [[[631], ..., [1517, ..., 1516]], ...] type='100 * var * var * uint32'>

In [78]:
ak.to_dataframe(reader.fileDir["superclustering/recoSuperCluster_constituentTs"].array(), anonymous="ts_id",
            levelname=lambda x : {0:"eventInternal", 1:"supercls_id", 2:"ts_in_supercls_id"}[x])

ts_id
eventInternal supercls_id ts_in_supercls_id       
0             0           0                    631
              1           0                   1812
                          1                   1691
              2           0                    582
              3           0                    453
...                                            ...
99            10          1                   2148
              11          0                    607
                          1                    509
                          2                    609
                          3                    369

[3148 rows x 1 columns]

In [71]:
reader.supercluster_all_df

ts_id
eventInternal supercls_id ts_in_supercls_id       
0             0           0                   1812
                          1                   1483
              1           0                     13
              2           0                     90
              3           0                    223
...                                            ...
99            43          0                   2202
              44          0                   2239
              45          0                   2242
              46          0                   2632
              47          0                   2747

[4804 rows x 1 columns]

In [ ]:
getCPToSuperclusterProperties(reader.supercluster_all_df, reader.assocs_bestScore_simToReco_df, 
        reader.tracksters_zipped[["ts_id", "raw_energy", "raw_em_energy", "regressed_energy", "raw_pt", "barycenter_eta"]], reader.simTrackstersCP_df)

In [60]:
df_reader = FWLiteDataframesReader("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v14-dnn/dataframes", 1, reader)
df_reader.supercls

ntuple  event_  ts_id
eventInternal supercls_id ts_in_supercls_id                       
0             0           0                       1       5   1812
                          1                       1       5   1483
              1           0                       1       5     13
              2           0                       1       5     90
              3           0                       1       5    223
...                                             ...     ...    ...
97            45          0                       1     100   2009
              46          0                       1     100   2034
              47          0                       1     100   2400
              48          0                       1     100   2589
              49          0                       1     100   2659

[4804 rows x 3 columns]

In [52]:
map_eventNb = ak.to_dataframe(reader.tracksters.event_, levelname=lambda x:"eventInternal", anonymous="event_")#.reset_index()
map_eventNb

,event_
eventInternal,
0,5
1,4
2,3
3,6
4,2
...,...
95,96
96,95
97,100


In [53]:
dnn_supercls_df = pd.read_pickle("/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v14-dnn/dataframes/supercls_1_test.pkl.gz")#.set_index(["event_", "supercls_id", "ts_in_supercls_id"])
dnn_supercls_df = pd.merge(dnn_supercls_df, map_eventNb.reset_index(), on="event_").set_index(["eventInternal", "supercls_id", "ts_in_supercls_id"])
dnn_supercls_df

ntuple  event_  ts_id
eventInternal supercls_id ts_in_supercls_id                       
0             0           0                       1       5   1812
                          1                       1       5   1483
              1           0                       1       5     13
              2           0                       1       5     90
              3           0                       1       5    223
...                                             ...     ...    ...
97            45          0                       1     100   2009
              46          0                       1     100   2034
              47          0                       1     100   2400
              48          0                       1     100   2589
              49          0                       1     100   2659

[4804 rows x 3 columns]

In [54]:
reader.supercluster_all_df

ts_id
eventInternal supercls_id ts_in_supercls_id       
0             0           0                   1812
                          1                   1483
              1           0                     13
              2           0                     90
              3           0                    223
...                                            ...
99            43          0                   2202
              44          0                   2239
              45          0                   2242
              46          0                   2632
              47          0                   2747

[4804 rows x 1 columns]

In [55]:
reader.assocs_bestScore_simToReco_df

ts_id     score     sharedE
eventInternal caloparticle_id                             
0             0                  631  0.002861  219.442230
              1                 1812  0.012030  191.803116
1             0                  739  0.002568  510.421753
              1                 2088  0.002458  499.531616
2             0                  954  0.318788   21.164497
...                              ...       ...         ...
97            1                 1982  0.006787  179.075974
98            0                  634  0.004153  392.520325
              1                 1838  0.005110  382.302979
99            0                  771  0.005811  343.893097
              1                 2186  0.011907  312.984100

[200 rows x 3 columns]

In [56]:
getCPToSuperclusterProperties(reader.supercluster_all_df, reader.assocs_bestScore_simToReco_df, 
        reader.tracksters_zipped[["ts_id", "raw_energy", "raw_em_energy", "regressed_energy", "raw_pt", "barycenter_eta"]], reader.simTrackstersCP_df)

supercls_id  raw_energy_seed  \
eventInternal caloparticle_id                                 
0             1                          0       200.929672   
              0                         17       228.426239   
1             1                          0       516.890320   
              0                         16       531.369995   
2             0                          0        23.194902   
...                                    ...              ...   
97            1                         43       185.403992   
98            1                          0       409.746124   
              0                         13       414.719421   
99            1                          0       330.272614   
              0                          1       359.003876   

                               raw_em_energy_seed  regressed_energy_seed  \
eventInternal caloparticle_id                                              
0             1                        200.547134             208.893906   
              0                        228.426239             242.749146   
1             1                        516.629639             539.893311   
              0                        530.946960             554.294312   
2             0                         23.194902              27.848148   
...                                           ...                    ...   
97            1                        185.403992             195.973923   
98            1                        408.856323             434.381775   
              0                        414.072601             436.465057   
99            1                        330.272614             348.384369   
              0                        359.003876             378.302063   

                               raw_pt_seed  barycenter_eta_seed  \
eventInternal caloparticle_id                                     
0             1                  39.390762             2.336010   
              0                  42.023884            -2.336827   
1             1                  87.493629             2.464795   
              0                  87.436203            -2.465089   
2             0                   6.907556            -1.801838   
...                                    ...                  ...   
97            1                  53.003685             1.924102   
98            1                  56.074509             2.635895   
              0                  57.555271            -2.636469   
99            1                  80.268158             2.072622   
              0                  88.737862            -2.072991   

                               raw_energy_supercls_sum  \
eventInternal caloparticle_id                            
0             1                             210.164322   
              0                             228.426239   
1             1                             523.075500   
              0                             531.369995   
2             0                              35.736172   
...                                                ...   
97            1                             185.403992   
98            1                             411.837891   
              0                             414.719421   
99            1                             360.068054   
              0                             376.054810   

                               raw_em_energy_supercls_sum  \
eventInternal caloparticle_id                               
0             1                                209.781784   
              0                                228.426239   
1             1                                522.814819   
              0                                530.946960   
2             0                                 35.736172   
...                                                   ...   
97            1                                185.403992   
98            1                               

In [57]:
getCPToSuperclusterProperties(dnn_supercls_df, reader.assocs_bestScore_simToReco_df, 
        reader.tracksters_zipped[["ts_id", "raw_energy", "raw_em_energy", "regressed_energy", "raw_pt", "barycenter_eta"]], reader.simTrackstersCP_df)

supercls_id  raw_energy_seed  \
eventInternal caloparticle_id                                 
0             1                          0       200.929672   
              0                         17       228.426239   
1             1                          0       516.890320   
              0                         16       531.369995   
2             0                          0        23.194902   
...                                    ...              ...   
97            1                         43       185.403992   
98            1                          0       409.746124   
              0                         13       414.719421   
99            1                          0       330.272614   
              0                          1       359.003876   

                               raw_em_energy_seed  regressed_energy_seed  \
eventInternal caloparticle_id                                              
0             1                        200.547134             208.893906   
              0                        228.426239             242.749146   
1             1                        516.629639             539.893311   
              0                        530.946960             554.294312   
2             0                         23.194902              27.848148   
...                                           ...                    ...   
97            1                        185.403992             195.973923   
98            1                        408.856323             434.381775   
              0                        414.072601             436.465057   
99            1                        330.272614             348.384369   
              0                        359.003876             378.302063   

                               raw_pt_seed  barycenter_eta_seed  \
eventInternal caloparticle_id                                     
0             1                  39.390762             2.336010   
              0                  42.023884            -2.336827   
1             1                  87.493629             2.464795   
              0                  87.436203            -2.465089   
2             0                   6.907556            -1.801838   
...                                    ...                  ...   
97            1                  53.003685             1.924102   
98            1                  56.074509             2.635895   
              0                  57.555271            -2.636469   
99            1                  80.268158             2.072622   
              0                  88.737862            -2.072991   

                               raw_energy_supercls_sum  \
eventInternal caloparticle_id                            
0             1                             210.164322   
              0                             228.426239   
1             1                             523.075500   
              0                             531.369995   
2             0                              35.736172   
...                                                ...   
97            1                             185.403992   
98            1                             411.837891   
              0                             414.719421   
99            1                             360.068054   
              0                             376.054810   

                               raw_em_energy_supercls_sum  \
eventInternal caloparticle_id                               
0             1                                209.781784   
              0                                228.426239   
1             1                                522.814819   
              0                                530.946960   
2             0                                 35.736172   
...                                                   ...   
97            1                                185.403992   
98            1                               